In [4]:
import os
import re
import time
import json
import random
from datetime import datetime
from pprint import pprint
from pathlib import Path
from collections import defaultdict
from functools import partial
from tqdm.notebook import tqdm

from urllib.parse import urlencode, urlparse, urlunparse, parse_qs, unquote

import pandas as pd
import numpy as np
import networkx as nx

from omegaconf import OmegaConf

from transformations.id import IDRules

import ipyplot as iplt
import matplotlib.pyplot as plt

pd.options.display.max_columns = 150
pd.options.display.max_rows = 100

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
data_dir = Path("./data/")

conf = {}
for merchant_conf_file in Path("./conf/merchants").glob("*.yaml"):
    conf[merchant_conf_file.stem] = OmegaConf.load(merchant_conf_file)
print(len(conf))

49


In [6]:
dfs = {}
(data_dir / "raw").mkdir(exist_ok=True, parents=True)

for merchant in tqdm(list(conf.keys())):
    try:
        feed_url = conf[merchant].feed.url
        feed_path = data_dir / "raw" / f"{merchant}.parquet"
        if feed_path.is_file():
            df = pd.read_parquet(feed_path)
        else:
            df = pd.read_csv(feed_url, compression=conf[merchant].feed.compression, low_memory=False)
            df["merchant"] = merchant
            df["timestamp"] = datetime.now()
            df.to_parquet(data_dir / "raw" / f"{merchant}.parquet")
        print(merchant, df.shape)
        dfs[merchant] = df
    except Exception as e:
        print(merchant, e)


  0%|          | 0/49 [00:00<?, ?it/s]

adidas (77570, 30)
allsole (2137, 94)
asos HTTP Error 400: Bad Request
begg_shoes (14568, 33)
blue_tomato (12675, 88)
brother2brother (5408, 24)
camper (10013, 94)
cho (11493, 24)
choice (11413, 94)
coggles (20757, 94)
converse (7262, 94)
daniel_footwear (4562, 36)
deichmann (1059, 23)
ellesse (4041, 88)
eqvvs (42193, 94)
eqvvs_women (3413, 88)
excell_sports (29078, 24)
footasylum (20487, 94)
footpatrol (5238, 88)
foot_locker (22405, 94)
hip (16002, 88)
hoka (5647, 94)
inov8 (1751, 94)
jd_sports (38926, 94)
kappa (7397, 94)
klekt (28423, 24)
mr_porter (68480, 23)
new_balance (11772, 94)
nike (56773, 94)
office (3694, 94)
offspring (464, 88)
otrium (110235, 23)
reebok (21818, 30)
schuh (23785, 88)
scotts (15726, 94)
secret_sales HTTP Error 400: Bad Request
sevenstore (4322, 94)
size (15210, 94)
slam_jam (12732, 28)
stadium_goods (52522, 26)
standout (11987, 94)
stuarts (11784, 94)
studio (74637, 88)
tessuti (34725, 94)
the_sports_edit (1171, 94)
tower_london (7530, 94)
under_armour (183

In [4]:
# merchant = "camper"
# df = dfs[merchant]

# for col in df.columns:
#     print("-" * 100)
#     print(col)
#     print("-" * 100)
#     print(df[col].value_counts().head(20))

In [5]:
# iplt.plot_images(df[df.parent_product_id == random.choice(df.parent_product_id)].merchant_image_url.values)

In [6]:
(data_dir / "filtered").mkdir(exist_ok=True, parents=True)


def filter_categories(df, conf):
    if not conf.filters is None:
        filters = OmegaConf.to_container(conf.filters)
        if isinstance(filters, dict):
            filters = [filters]
        assert isinstance(filters, list)
        for cat_filter in filters:
            cat_col = cat_filter["col"]
            include = 1
            if "include" in cat_filter:
                cat_val = cat_filter["include"]
            elif "exclude" in cat_filter:
                include = 0
                cat_val = cat_filter["exclude"]

            if isinstance(cat_val, str):
                cat_val = [cat_val]
            assert isinstance(cat_val, list)

            # print("category col:", cat_col)
            # print(df[cat_col].value_counts().head(20))
            # print()
            # print("rows before: ", len(df))
            if include == 1:
                # print("include: ", cat_val)
                df = df.loc[df[cat_col].isin(set(cat_val))].reset_index(drop=True)
            elif include == 0:
                # print("exclude: ", cat_val)
                df = df.loc[~df[cat_col].isin(set(cat_val))].reset_index(drop=True)
            # print("rows after: ", len(df))
            # print()
            # print("cols retained:")
            # print(df[cat_col].value_counts())
    return df


for merchant in tqdm(dfs):
    df = dfs[merchant]
    dfs[merchant] = filter_categories(df, conf[merchant])
    df.to_parquet(data_dir / "filtered" / f"{merchant}.parquet")


  0%|          | 0/46 [00:00<?, ?it/s]

In [7]:
for merchant in tqdm(dfs):
    print(merchant, end=" : ")
    df = dfs[merchant]
    id_col = conf[merchant].id
    id_rule = IDRules(id_col, merchant)
    df = df.dropna(subset=[id_col]).reset_index(drop=True)
    df["id"] = df.apply(id_rule, axis=1)
    df = df.dropna(subset=["id"]).reset_index(drop=True)
    print(df.id.nunique())
    dfs[merchant] = df


  0%|          | 0/46 [00:00<?, ?it/s]

allsole : 381
begg_shoes : 2578
blue_tomato : 537
brother2brother : 192
camper : 1066
cho : 604
choice : 516
coggles : 1145
converse : 536
daniel_footwear : 1090
deichmann : 83
ellesse : 65
eqvvs : 239
eqvvs_women : 133
excell_sports : 753
footasylum : 1078
footpatrol : 935
foot_locker : 2832
hip : 860
hoka : 166
inov8 : 76
jd_sports : 1350
kappa : 11
klekt : 5918
mr_porter : 1062
new_balance : 1602
nike : 1988
office : 959
offspring : 81
otrium : 3759
reebok : 1213
schuh : 1543
scotts : 497
sevenstore : 380
size : 1685
slam_jam : 656
stadium_goods : 8291
standout : 349
stuarts : 935
studio : 604
tessuti : 1472
the_sports_edit : 65
tower_london : 1907
under_armour : 446
very : 5259
vrnts : 123


In [8]:
(data_dir / "transformed").mkdir(exist_ok=True, parents=True)

new_cols = set()
for merchant, df in tqdm(dfs.items()):
    # add merchant column
    df["merchant"] = merchant

    # apply column mapping
    col_map = conf[merchant].column_map
    for from_col, to_col in col_map.items():
        df[to_col] = df[from_col]
        new_cols.add(to_col)

    if "brand" not in df.columns:
        df["brand"] = merchant

    # extract codes
    for code_col, code in conf[merchant].get("codes", {}).items():
        df[code] = df[code_col]
        new_cols.add(code)

    dfs[merchant] = df
    df.to_parquet(data_dir / "transformed" / f"{merchant}.parquet")
print(sorted(new_cols))


  0%|          | 0/46 [00:00<?, ?it/s]

['ac', 'age', 'brand', 'category', 'color', 'description', 'gender', 'gtin', 'image_url', 'jd', 'material', 'merchant_deep_link', 'merchant_sku', 'pattern', 'price', 'product_name', 'purpose', 'sf', 'short_description', 'size', 'style']


In [9]:
match_cols = ["style", "gtin", "jd", "ac", "sf"]

for merchant in dfs:
    df = dfs[merchant]
    dfs[merchant] = df.loc[
        :,
        [
            col
            for col in [
                "id",
                "merchant",
                "product_name",
                "brand",
                "age",
                "gender",
                "category",
                "merchant_sku",
                "short_description",
                "description",
                "merchant_deep_link",
                "price",
                "pattern",
                "material",
                "purpose",
                "color",
                *match_cols,
                "image_url",
            ]
            if col in df.columns
        ],
    ]


In [10]:
df = (
    pd.concat(dfs.values(), ignore_index=True)
    .drop_duplicates("id")
    .reset_index(drop=True)
)
df


,id,merchant,product_name,brand,age,gender,category,merchant_sku,short_description,description,merchant_deep_link,price,material,color,style,gtin,ac,image_url,pattern,sf,jd,purpose
0,allsole.12104416,allsole,Barbour Men's Nelson Nubuck 3-Eye Chukka Boots...,Barbour,adult,male,Boots,12104416,Barbour brown leather mudguard chukka boots wi...,Barbour brown leather mudguard chukka boots wi...,https://www.allsole.com/boots-clothing-men-foo...,GBP88.00,Upper: Leather/Nylon. Sole: EVA.,Tan,MFO0386BR91,NaN,12104416.0,https://s4.thcdn.com/productimg/1600/1600/1210...,NaN,NaN,NaN,NaN
1,allsole.11721822,allsole,Barbour Men's Readhead Leather Chukka Boots - ...,Barbour,adult,male,Boots,11721822,Men’s ‘Readhead’ chukka boots from iconic Brit...,Men’s ‘Readhead’ chukka boots from iconic Brit...,https://www.allsole.com/boots-clothing-men-foo...,GBP129.00,Upper: Leather. Lining: Cotton/Leather. Sole: ...,Black,MFO0138BK11,NaN,11721822.0,https://s4.thcdn.com/productimg/1600/1600/1172...,NaN,NaN,NaN,NaN
2,allsole.12104410,allsole,Barbour Men's Farsley Leather Chelsea Boots - ...,Barbour,adult,male,Boots,12104410,Black leather Barbour Chelsea boots with a cot...,Black leather Barbour Chelsea boots with a cot...,https://www.allsole.com/boots-clothing-men-foo...,GBP129.00,Upper: Leather. Lining: Cotton/Leather. Sole: ...,Black,MFO0244BK11,NaN,12104410.0,https://s4.thcdn.com/productimg/1600/1600/1210...,NaN,NaN,NaN,NaN
3,allsole.10667310,allsole,Barbour Men's Readhead Chukka Boots - Tan,Barbour,adult,male,Boots,10667310,Barbour tan leather chukka boots with contrast...,Barbour tan leather chukka boots with contrast...,https://www.allsole.com/boots-clothing-men-foo...,GBP129.00,Upper: Leather. Lining: Cotton/Leather. Sole: ...,Tan,MFO0138TA72,NaN,10667310.0,https://s4.thcdn.com//productimg/1600/1600/106...,NaN,NaN,NaN,NaN
4,allsole.11912702,allsole,Barbour Men's Toeman Beach Toe Post Sandals - ...,Barbour,adult,male,Sandals,11912702,Barbour sandals with a woven toe post strap an...,Barbour sandals with a woven toe post strap an...,https://www.allsole.com/sandals-men-footwear/b...,GBP25.95,Upper: Synthetic/Textile. Sole: Synthetic.,Blue,MBS0007NY98,NaN,11912702.0,https://s4.thcdn.com/productimg/1600/1600/1191...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58015,vrnts.CMBA009S22FLE0011025,vrnts,MARCELO BURLON Icon Wings sweatshirt,Marcelo Burlon,NaN,Male,Shoes > Men's Shoes > Sneakers > NIKE,CMBA009S22FLE0011025,Marcelo Burlon County of Milan Icon Wings swea...,Marcelo Burlon County of Milan Icon Wings swea...,NaN,305.0,NaN,Black,CMBA009S22FLE0011025,8058841396815.0,NaN,http://media.vrients.com/media/catalog/product...,NaN,NaN,NaN,NaN
58016,vrnts.CMBB007S22FLE0041025,vrnts,MARCELO BURLON Solfolk Wings hoodie,Marcelo Burlon,NaN,Male,Shoes > Men's Shoes > Sneakers > NIKE,CMBB007S22FLE0041025,Marcelo Burlon County of Milan Solfolk Wings h...,Marcelo Burlon County of Milan Solfolk Wings h...,NaN,330.0,NaN,Black,CMBB007S22FLE0041025,8058841405388.0,NaN,http://media.vrients.com/media/catalog/product...,NaN,NaN,NaN,NaN
58017,vrnts.CMCI010C99FLE0011001,vrnts,MARCELO BURLON Cross basket shorts,Marcelo Burlon,NaN,Male,Shoes > Men's Shoes > Sneakers > NIKE,CMCI010C99FLE0011001,Marcelo Burlon County of Milan Cross basket sh...,Marcelo Burlon County of Milan Cross basket sh...,NaN,230.0,NaN,Black,CMCI010C99FLE0011001,8058841389633.0,NaN,http://media.vrients.com/media/catalog/product...,NaN,NaN,NaN,NaN
58018,vrnts.CMFA003S22FAB0011084,vrnts,MARCELO BURLON Cross swim shorts,Marcelo Burlon,NaN,Male,Shoes > Men's Shoes > Sneakers > NIKE,CMFA003S22FAB0011084,Marcelo Burlon County of Milan Cross swim shorts,Marcelo Burlon County of Milan Cross swim shor...,NaN,180.0,NaN,Black,CMFA003S22FAB0011084,8058841411877.0,NaN,http://media.vrients.com/media/catalog/product...,NaN,NaN,NaN,NaN


In [11]:
for col in match_cols:
    print(col)
    print(df.groupby(col).merchant.nunique().value_counts())

style
1    23439
2     1985
3      310
4       54
5        4
Name: merchant, dtype: int64
gtin
1    40369
2     1400
3      178
4       26
5        7
6        2
Name: merchant, dtype: int64
jd
1    3405
2     342
3      30
Name: merchant, dtype: int64
ac
1    1210
2     158
Name: merchant, dtype: int64
sf
1    1458
Name: merchant, dtype: int64


In [12]:
import itertools

G = nx.Graph()
col = "gtin"
df.groupby(col).id.unique().apply(partial(itertools.combinations, r=2)).apply(
    list
).apply(G.add_edges_from)
G.number_of_nodes(), G.number_of_edges(), len(list(nx.connected_components(G)))

(5430, 5734, 2230)